<a href="https://colab.research.google.com/github/IRGarrido/problema_mochila_repo/blob/master/MochilaREPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aplicação do Problema da Mochila com o jogo R.E.P.O

Autor: Ian Garrido Reis - reisiangarrido@gmail.com

R.E.P.O é um jogo de terror cooperativo em que o jogador é um robô que deve sobreviver a monstros enquanto coleta itens de valor (*valuables*)  em um cenário abandonado. No entanto, os ***Extraction points*** tem um tamanho delimitado em que os itens devem ser alocados para não serem destruídos. Além disso, o jogador deve conduzir os itens até o ponto de extração sem colidí-lo, do contrário o valor é reduzido. De tal forma, deve-se buscar uma razão entre o valor e tamanho do item a fim de minimizar a dificuldade.

Utiliza-se como fonte o site [Repo Wiki | Fandom](https://repo-2025horror.fandom.com/wiki/Valuables) que enumera os valores em 11 categorias:

*   Very cheap (Below $300)

*   Cheap- ($300-$450)
*   Cheap ($500-$650)
*   Cheap+ ($850-$1100)
*   Cheap++ ($1200-$2000)
*   Medium ($2000-$3000)
*   Medium+ ($3500-$4500)
*   High ($5500-$7500)
*   High+ ($9500-$12000)
*   Expensive ($18000-$25000)
*   Expensive+ ($30000-$45000)

Os tamanhos são agrupados 7 categorias:

*   Tiny
*   Small
*   Medium
*   Big
*   Wide
*   Tall
*   Very Tall

___


## Importação de Bibliotecas e Dataset

In [19]:
import pandas as pd
import numpy as np
from prettytable import PrettyTable

In [20]:
url = "https://repo-2025horror.fandom.com/wiki/Valuables"

tabelas = pd.read_html(url)

df = pd.DataFrame(tabelas[0])

items = df[['Item Name', 'Value', 'Size']]
items

,Item Name,Value,Size
0,Diamond,Medium,Tiny
1,Emerald Ring,Cheap,Tiny
2,Goblet,Cheap,Tiny
3,Ocarina,Cheap,Tiny
4,Pocket Watch,Cheap,Tiny
...,...,...,...
84,Blender,Expensive,Tall?
85,Server Rack,Expensive,Very Tall
86,Golden Statue,Expensive,Very Tall
87,Grandfather Clock,Expensive,Very Tall


## Levantamento de informações

In [21]:
items.describe()

,Item Name,Value,Size
count,89,89,89
unique,89,15,12
top,Diamond,Medium,Medium
freq,1,21,19


In [22]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Item Name  89 non-null     object
 1   Value      89 non-null     object
 2   Size       89 non-null     object
dtypes: object(3)
memory usage: 2.2+ KB


In [23]:
items.dropna()

,Item Name,Value,Size
0,Diamond,Medium,Tiny
1,Emerald Ring,Cheap,Tiny
2,Goblet,Cheap,Tiny
3,Ocarina,Cheap,Tiny
4,Pocket Watch,Cheap,Tiny
...,...,...,...
84,Blender,Expensive,Tall?
85,Server Rack,Expensive,Very Tall
86,Golden Statue,Expensive,Very Tall
87,Grandfather Clock,Expensive,Very Tall


##Manipulação de Colunas

###Tamanho

In [24]:
sizes = items['Size'].unique()
sizes

array(['Tiny', 'Tiny?', 'Small?', 'Small', 'Medium?', 'Medium', 'Big?',
       'Big', 'Wide', 'Tall', 'Tall?', 'Very Tall'], dtype=object)

In [25]:
size_values = {
    'Tiny': 1,
    'Tiny?': 1,
    'Small?': 5,
    'Small': 5,
    'Medium?': 50,
    'Medium': 50,
    'Big?': 250,
    'Big': 250,
    'Wide': 300,
    'Tall': 300,
    'Tall?': 300,
    'Very Tall': 500
}

In [26]:
for size, value in size_values.items():
    items.replace(size, value, inplace=True)

<ipython-input-26-e85824f2f071>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  items.replace(size, value, inplace=True)
<ipython-input-26-e85824f2f071>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  items.replace(size, value, inplace=True)


###Valor

In [27]:
sizes = items['Value'].unique()
sizes

array([50, 'Cheap', 'Cheap++', 'Cheap+', 'Medium+', '$1k-$3k', '$1-$5000',
       'High', '$3k-$5k', '$5001-$10000', 'High+', '$5k-$8k', '$10001+',
       'Expensive+', 'Expensive'], dtype=object)

In [28]:
price_categories = {
    "Cheap": 650,
    "Cheap+": 1100,
    "Cheap++": 2000,
    "Medium": 3000,
    "Medium+": 4500,
    "High": 7500,
    "High+": 12000,
    "Expensive": 25000,
    "Expensive+": 45000,
    "$1k-$3k" : 3000,
    "$1-$5000": 5000,
    "$3k-$5k": 5000,
    "$5001-$10000": 10000,
    "$5k-$8k": 8000,
    "$10001+": 10001,
}

In [29]:
for category, price in price_categories.items():
    items.replace(category, price, inplace=True)

<ipython-input-29-da2674779b9d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  items.replace(category, price, inplace=True)
<ipython-input-29-da2674779b9d>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  items.replace(category, price, inplace=True)


In [30]:
items.head(20)

,Item Name,Value,Size
0,Diamond,50,1
1,Emerald Ring,650,1
2,Goblet,650,1
3,Ocarina,650,1
4,Pocket Watch,650,1
5,Uranium Mug,2000,1
6,Tooth,650,1
7,Gold Fish,50,1
8,Banana Bow,1100,5
9,Silver Fish,1100,1


#Problema da Mochila

O problema da mochila (em inglês, knapsack problem) é um problema clássico de otimização combinatória. Nesse caso, estamos utilizando a versão 0-1, em que um dado elemento pode ser inserido ou não na mochila sem que seja fracionado. Por ser um problema NP Completo, existem diferentes abordagens para obter uma solução para o problema, algumas são apresentadas a seguir.

In [31]:
NUM_EP = 1
EP_SIZE = 1500
MAX_SIZE = NUM_EP * EP_SIZE

In [32]:
items['Rate'] = items['Value'] / items['Size']
items.head()

<ipython-input-32-b232a09b1f2c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  items['Rate'] = items['Value'] / items['Size']


,Item Name,Value,Size,Rate
0,Diamond,50,1,50.0
1,Emerald Ring,650,1,650.0
2,Goblet,650,1,650.0
3,Ocarina,650,1,650.0
4,Pocket Watch,650,1,650.0


##Algoritmo Guloso

Ordenação de Acordo com a razão Valor/Tamanho

In [33]:
df = items.copy()
df = df.sort_values(by=['Rate'], ascending=False).reset_index(drop=True)
df

,Item Name,Value,Size,Rate
0,Uranium Mug,2000,1,2000.0
1,Cool Brain,1100,1,1100.0
2,Toast,1100,1,1100.0
3,Silver Fish,1100,1,1100.0
4,Small Surplus Valuable,5000,5,1000.0
...,...,...,...,...
84,Sample,50,50,1.0
85,Clown,50,50,1.0
86,Power Crystal,50,50,1.0
87,Computer,50,50,1.0


In [34]:
current_size = 0
current_value = 0
num_items = 0

table = PrettyTable()
table.field_names = ["Iteração", "Item", "Tamanho", "Valor Somado", "Preenchimento"]

while(current_size < MAX_SIZE):

    if df.size == 0:
        break

    elif current_size + df.loc[0, 'Size'] <= MAX_SIZE:
        current_size += df.loc[0, 'Size']
        current_value += df.loc[0, 'Value']
        num_items += 1
        table.add_row([num_items, df.loc[0, 'Item Name'], df.loc[0, 'Size'], current_value, f"{current_size}/{MAX_SIZE}"])

        df = df.iloc[1:].reset_index(drop=True)

    else:
        df = df.iloc[1:].reset_index(drop=True)
        continue

print(table)

print(f"Nº de Itens: {num_items}")
print(f"Valor Total: {current_value}")
print(f"Tamanho Total: {current_size}")

+----------+-------------------------+---------+--------------+---------------+
| Iteração |           Item          | Tamanho | Valor Somado | Preenchimento |
+----------+-------------------------+---------+--------------+---------------+
|    1     |       Uranium Mug       |    1    |     2000     |     1/1500    |
|    2     |        Cool Brain       |    1    |     3100     |     2/1500    |
|    3     |          Toast          |    1    |     4200     |     3/1500    |
|    4     |       Silver Fish       |    1    |     5300     |     4/1500    |
|    5     |  Small Surplus Valuable |    5    |    10300     |     9/1500    |
|    6     |        Music Box        |    5    |    14800     |    14/1500    |
|    7     |       Pocket Watch      |    1    |    15450     |    15/1500    |
|    8     |         Ocarina         |    1    |    16100     |    16/1500    |
|    9     |       Emerald Ring      |    1    |    16750     |    17/1500    |
|    10    |        Ruben Doll       |  

##Programação Dinâmica

In [35]:
n = len(items)
df = items.copy()
dp = [[0 for _ in range(MAX_SIZE + 1)] for _ in range(n + 1)]

for i in range(1, n + 1):
    size = df.loc[i - 1, 'Size']
    value = df.loc[i - 1, 'Value']
    for s in range(1, MAX_SIZE + 1):
        if size <= s:
            dp[i][s] = max(dp[i - 1][s], dp[i - 1][s - size] + value)
        else:
            dp[i][s] = dp[i - 1][s]

solucao = [0] * n
s = MAX_SIZE

for i in range(n, 0, -1):
    if dp[i][s] != dp[i-1][s]:
        solucao[i-1] = 1
        s -= items.loc[i-1, 'Size']

table = PrettyTable()
table.field_names = ["Iteração", "Item", "Tamanho", "Valor Somado", "Preenchimento"]

num_items = 0
sum_sizes = 0
for i in range(n):
    if solucao[i] == 1:
        num_items += 1
        sum_sizes += df.loc[i, 'Size']
        table.add_row([num_items, df.loc[i, 'Item Name'], df.loc[i, 'Size'], dp[i+1][sum_sizes], f"{sum_sizes}/{MAX_SIZE}"])
print(table)
print("Nº de Itens:", num_items)
print("Valor máximo:", dp[n][MAX_SIZE])
print("Tamanho máximo:", sum_sizes)

+----------+-------------------------+---------+--------------+---------------+
| Iteração |           Item          | Tamanho | Valor Somado | Preenchimento |
+----------+-------------------------+---------+--------------+---------------+
|    1     |       Emerald Ring      |    1    |     650      |     1/1500    |
|    2     |          Goblet         |    1    |     1300     |     2/1500    |
|    3     |         Ocarina         |    1    |     1950     |     3/1500    |
|    4     |       Pocket Watch      |    1    |     2600     |     4/1500    |
|    5     |       Uranium Mug       |    1    |     4600     |     5/1500    |
|    6     |          Tooth          |    1    |     5250     |     6/1500    |
|    7     |        Banana Bow       |    5    |     6350     |    11/1500    |
|    8     |       Silver Fish       |    1    |     7450     |    12/1500    |
|    9     |        Cool Brain       |    1    |     8550     |    13/1500    |
|    10    |        Ruben Doll       |  